In [6]:
import pandas as pd 
import numpy as np 
import re
from openai import OpenAI
import ast
from tqdm import tqdm 
import matplotlib.pyplot as plt 
import plotly.express as px
import statsmodels.formula.api as smf
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr, pearsonr
import pickle
import itertools
import os
client = OpenAI(api_key=open("/Users/mosobay/Dropbox (MIT)/research/openai_api", "r").read().strip())

In [37]:
def parse_response(response):
    try:
        return ast.literal_eval(response)
    except:
        return None

In [46]:
def clean_task_html(text):
    pattern = r"(?<=Set-Up)(.*)"
    match = re.search(pattern, text)
    extracted_text = match.group(1)
    no_html = re.sub(r"<[^>]+>", "", extracted_text)
    return no_html 

In [58]:
df_tasks = pd.read_csv("/Users/mosobay/Dropbox (MIT)/research/homo_silicus/task_htmls.csv")
df_tasks["task_desc"] = [clean_task_html(x) for x in df_tasks["html"]]

In [57]:
df_pairwise = pd.concat([pd.DataFrame(pickle.load(open(f"../data/pairwise_comparisons/{x}", "rb"))).assign(ref_task = x.split("_")[1][:2]) for x in os.listdir("../data/pairwise_comparisons/") if "pkl" in x],
                        ignore_index=True)

df_pairwise["response_parsed"] = [parse_response(x.choices[0].message.content) for x in df_pairwise["gpt_response"]]
df_pairwise = df_pairwise.dropna(subset="response_parsed").reset_index(drop=True)

In [64]:
df_pairwise["rationale"] = [x["rationale"] for x in df_pairwise["response_parsed"]]
df_pairwise["winning_task"] = [x["choice"] for x in df_pairwise["response_parsed"]]

df_pairwise["ref_task_desc"] = df_tasks["task_desc"].loc[df_pairwise["ref_task"].astype(int).values].values
df_pairwise["ref_task_name"] = df_tasks["task"].loc[df_pairwise["ref_task"].astype(int).values].values

df_pairwise["a_task_desc"] = df_tasks["task_desc"].loc[df_pairwise["A"].astype(int).values].values
df_pairwise["a_task_name"] = df_tasks["task"].loc[df_pairwise["A"].astype(int).values].values

df_pairwise["b_task_desc"] = df_tasks["task_desc"].loc[df_pairwise["B"].astype(int).values].values
df_pairwise["b_task_name"] = df_tasks["task"].loc[df_pairwise["B"].astype(int).values].values

In [67]:
df_pairwise[["ref_task_name", "ref_task_desc", "a_task_name", "a_task_desc", "b_task_name", "b_task_desc", "winning_task", "rationale"]]

,ref_task_name,ref_task_desc,a_task_name,a_task_desc,b_task_name,b_task_desc,winning_task,rationale
0,Advertisement writing,Participants were given the description of a p...,New Recruit,Participants are randomly paired for a negotia...,Blocks World for Teams,Participants work together in a virtual world ...,B,Task B is more similar to the reference task i...
1,Advertisement writing,Participants were given the description of a p...,Investment Game (aka Trust Game),Participants are assigned to one of two rooms:...,Logic Problem,"Participants receive a logic problem, which is...",B,Task B is more similar to the reference task i...
2,Advertisement writing,Participants were given the description of a p...,Checkers,Participants are given an online checkers boar...,Chicken,The game of Chicken simulates a potential conf...,A,Task A involves participants working together ...
3,Advertisement writing,Participants were given the description of a p...,Architectural design task,Participants receive written and video instruc...,Graph coloring task,"Participants are given one or more ""graph colo...",A,Task A is more similar to the reference task i...
4,Advertisement writing,Participants were given the description of a p...,New Recruit,Participants are randomly paired for a negotia...,Bullard Houses,Participants are assigned to play in one of th...,B,Task B is more similar to the reference task i...
...,...,...,...,...,...,...,...,...
2357,Apache helicopter flight simulator (Longbow2),Participants are randomly assigned to groups o...,Husbands and wives transfer,Participants are given a problem that reads as...,Intergroup Prisoner's Dilemma,All participants are given a fixed amount of i...,A,Task A is more similar to the reference task i...
2358,Apache helicopter flight simulator (Longbow2),Participants are randomly assigned to groups o...,Minimum-effort tacit coordination game,Participants play in a large group with others...,Reading the mind in the eyes,Participants were given 36 pairs of eyes. Each...,B,Task B is more similar to the reference task i...
2359,Apache helicopter flight simulator (Longbow2),Participants are randomly assigned to groups o...,Sender-Receiver game,Participants are paired for a communication-ba...,Minimum-effort tacit coordination game,Participants play in a large group with others...,B,Task B is more similar to the reference task i...
2360,Apache helicopter flight simulator (Longbow2),Participants are randomly assigned to groups o...,Aerospace Investment (Role-playing),Participants are randomly paired for a scored ...,Shopping plan,Each participant has a grocery list and a map ...,B,Task B is more similar to the reference task i...


In [60]:
df_tasks["task_desc"].loc[df_pairwise["ref_task"].astype(int).values]

17    Participants were given the description of a p...
17    Participants were given the description of a p...
17    Participants were given the description of a p...
17    Participants were given the description of a p...
17    Participants were given the description of a p...
                            ...                        
87    Participants are randomly assigned to groups o...
87    Participants are randomly assigned to groups o...
87    Participants are randomly assigned to groups o...
87    Participants are randomly assigned to groups o...
87    Participants are randomly assigned to groups o...
Name: task_desc, Length: 2362, dtype: object

In [52]:
df_tasks

,task,html,github,path,script,task_desc
0,Euclidean traveling salesperson,<h2>Euclidean traveling salesperson</h2><br><h...,https://api.github.com/repos/Watts-Lab/task-ma...,tasks/summaries/Euclidean traveling salesperso...,https://api.github.com/repos/Watts-Lab/task-ma...,"Participants get a map with multiple cities, w..."
1,Moral Reasoning (Disciplinary Action Case),<h2>Moral Reasoning (Disciplinary Action Case)...,https://api.github.com/repos/Watts-Lab/task-ma...,tasks/summaries/Moral Reasoning (Disciplinary ...,https://api.github.com/repos/Watts-Lab/task-ma...,"Participants read a fictitious, controversial ..."
2,To evacuate or not to evacuate,<h2>To evacuate or not to evacuate</h2><br><h2...,https://api.github.com/repos/Watts-Lab/task-ma...,tasks/summaries/To evacuate or not to evacuate...,https://api.github.com/repos/Watts-Lab/task-ma...,Participants read about a situation in which '...
3,Allocating resources to programs,<h2>Allocating resources to programs</h2><br><...,https://api.github.com/repos/Watts-Lab/task-ma...,tasks/summaries/Allocating resources to progra...,https://api.github.com/repos/Watts-Lab/task-ma...,Partipants are given a list of complex competi...
4,Shopping plan,<h2>Shopping plan</h2><br><h2>1. Set-Up</h2><p...,https://api.github.com/repos/Watts-Lab/task-ma...,tasks/summaries/Shopping plan-writeup.md,https://api.github.com/repos/Watts-Lab/task-ma...,Each participant has a grocery list and a map ...
...,...,...,...,...,...,...
97,Investment game (hidden-profile),<h2>Investment game (hidden-profile)</h2><br><...,https://api.github.com/repos/Watts-Lab/task-ma...,tasks/summaries/Investment game (hidden-profil...,https://api.github.com/repos/Watts-Lab/task-ma...,Participants are asked to take the role of the...
98,Dictator game and its variants,<h2>Dictator game and its variants</h2><br><h2...,https://api.github.com/repos/Watts-Lab/task-ma...,tasks/summaries/Dictator game and its variants...,https://api.github.com/repos/Watts-Lab/task-ma...,Participants are placed into pairs and randoml...
99,Intergroup Prisoner's Dilemma,<h2>Intergroup Prisoner's Dilemma</h2><br><h2>...,https://api.github.com/repos/Watts-Lab/task-ma...,tasks/summaries/Intergroup Prisoner's Dilemma-...,https://api.github.com/repos/Watts-Lab/task-ma...,All participants are given a fixed amount of i...
100,Chicken,<h2>Chicken</h2><br><h2>1. Set-Up</h2><p>The g...,https://api.github.com/repos/Watts-Lab/task-ma...,tasks/summaries/Chicken-writeup.md,https://api.github.com/repos/Watts-Lab/task-ma...,The game of Chicken simulates a potential conf...


In [33]:
ast.literal_eval(df_pairwise["gpt_response"][0].choices[0].message.content)

{'choice': 'B',
 'rationale': 'Task B is more similar to the reference task in terms of team dynamics because both tasks require participants to work collaboratively towards a common goal, which is to complete a task within a limited amount of time. In the reference task, participants work together to create a single collective write-up for an advertisement, while in Task B, participants work together to deliver the correct sequence of colored blocks to the drop zone. Both tasks require coordination, communication, and a shared strategy to achieve the objective. In contrast, Task A involves a negotiation scenario where participants have individual and potentially conflicting objectives, which is fundamentally different from the collaborative nature of the reference task and Task B.'}

[{'choice': 'B',
  'rationale': 'Task B is more similar to the reference task in terms of team dynamics because both tasks require participants to work collaboratively towards a common goal, which is to complete a task within a limited amount of time. In the reference task, participants work together to create a single collective write-up for an advertisement, while in Task B, participants work together to deliver the correct sequence of colored blocks to the drop zone. Both tasks require coordination, communication, and a shared strategy to achieve the objective. In contrast, Task A involves a negotiation scenario where participants have individual and potentially conflicting objectives, which is fundamentally different from the collaborative nature of the reference task and Task B.'},
 {'choice': 'B',
  'rationale': 'Task B is more similar to the reference task in terms of team dynamics because both tasks involve creative problem-solving and collaboration to produce a single solution